In [ ]:
from itertools import chain
from pathlib import Path
from google_photos_takeout_model.models.albums import Album
from google_photos_takeout_model.models.media_items import get_media_items
from devtools import pprint
from more_itertools import only
from more_itertools import first
from shutil import copy

In [ ]:
albums = Path("D:/Google takeout/2024-11-20-photos")
people = list((albums / "people").iterdir())
undated = list((albums / "undated").iterdir())
trash = albums / "trash"
print(
    f"{len(people)=}",
    f"{len(undated)=}",
    f"Trash: {trash.relative_to(albums).as_posix()}",
    sep="\n",
)

In [ ]:
all_years = albums / "years"
years = {year.name: get_media_items(year) for year in all_years.iterdir()}
all_media_items = list(chain.from_iterable(years.values()))
print(
    f"{len(years)=}",
    f"{len(all_media_items)=}",
    sep="\n",
)

In [ ]:
pprint(all_media_items[-1])

In [ ]:
dated = {
    album_path.name: Album.from_path(album_path)
    for album_path in [
        *(albums / "dated-done").iterdir(),
    ]
}
print(
    f"{len(dated)=}",
    sep="\n",
)

In [ ]:
album = dated["2008-(11-27) – (12-17) Snow Days"]
found_media_items_metadata_names = [
    media_item.metadata_path.name for media_item in album.media_items
]
all_media_items_metadata = list(album.path.glob("[!metadata]*.json"))
metadata_paths_without_media_items = [
    metadata_path
    for metadata_path in all_media_items_metadata
    if metadata_path.name not in found_media_items_metadata_names
]
only(metadata_paths_without_media_items, default=None)

In [ ]:
if metadata_paths_without_media_items:
    missing = metadata_paths_without_media_items[0]
    found = first(
        [
            media_item
            for media_item in all_media_items
            if missing.with_name(missing.stem).stem == media_item.path.stem
        ]
    )
    dest = album.path / found.path.name
    copy(found.path, dest)
    found.path = dest
    album.media_items.append(found)

In [ ]:
missing_media_items: dict[str, list[Path]] = {}
for name, album in dated.items():
    found_media_items_metadata_names = [
        media_item.metadata_path.name for media_item in album.media_items
    ]
    all_media_items_metadata = list(album.path.glob("[!metadata]*.json"))
    metadata_paths_without_media_items = [
        metadata_path
        for metadata_path in all_media_items_metadata
        if metadata_path.name not in found_media_items_metadata_names
    ]
    missing_media_items[name] = metadata_paths_without_media_items
    for missing in metadata_paths_without_media_items:
        found = first(
            [
                media_item
                for media_item in all_media_items
                if missing.with_name(missing.stem).stem == media_item.path.stem
            ]
        )
        dest = album.path / found.path.name
        copy(found.path, dest)
        found.path = dest
        album.media_items.append(found)
pprint(missing_media_items)